In [ ]:
import pandas as pd
import numpy as np
base_casas = pd.read_csv('house_prices.csv')
base_casas

In [ ]:
base_casas.drop(columns = ['id', 'date', 'sqft_living15', 'sqft_lot15'], axis = 1, inplace = True)
base_casas

In [ ]:
base_treinamento = base_casas.iloc[0:15129,:]
base_treinamento.shape

In [ ]:
base_teste = base_casas.iloc[15129:,:]
base_teste.shape

In [ ]:
X_teste = base_teste.iloc[:,1:17].values
X_teste

In [ ]:
y_teste = base_teste.iloc[:, 0].values
y_teste

In [ ]:
base_treinamento.to_csv('house_prices_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('house_prices_test_xgboost.csv', header = False, index = False)

# Configurações SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session

In [ ]:
session = sagemaker.Session()
bucket = 'cursoawssagemaker'
subpasta_modelo = 'modelos/house-prices/xgboost'
subpasta_dataset = 'datasets/house-prices'
key_train = 'houses-train-data-xgboost'
key_test = 'houses-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

In [ ]:
import os
with open('house_prices_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
with open('house_prices_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

# Treinamento do XGBoost

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
from sagemaker import image_uris
container = image_uris.retrieve(framework = 'xgboost', region=boto3.Session().region_name, version='latest')

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
xgboost = sagemaker.estimator.Estimator(image_uri = container,
                                        role = role,
                                        instance_count = 1,
                                        instance_type = 'ml.m5.2xlarge',
                                        output_path = output_location,
                                        sagemaker_session = session)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgboost.set_hyperparameters(num_round = 100)

In [ ]:
train_input = sagemaker.inputs.TrainingInput(s3_data = s3_train_data, content_type='csv', s3_data_type = 'S3Prefix')
validation_input = sagemaker.inputs.TrainingInput(s3_data = s3_test_data, content_type='csv', s3_data_type = 'S3Prefix')
data_channels = {'train': train_input, 'validation': validation_input}

In [ ]:
xgboost.fit(data_channels)